In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from einops import rearrange, repeat, pack, unpack, einsum

b_s = 16
seq_len = 512
head_dim = 10
num_attn_heads = 8
embed_dim = 13 


In [6]:
# Create some fake traininig data

input_data = torch.randn(b_s, seq_len, embed_dim)

Wq = nn.Linear(embed_dim, num_attn_heads * head_dim, bias=False)
Wk = nn.Linear(embed_dim, num_attn_heads * head_dim, bias=False)
Wv = nn.Linear(embed_dim, num_attn_heads * head_dim, bias=False)
Wo = nn.Linear(num_attn_heads * head_dim, embed_dim) # Look up why Wo can hava a bias term

queries = Wq(input_data)
keys = Wk(input_data)
values = Wv(input_data)

# The above is all the same, now we're goint to add fake recurrence

recurrence = torch.randn(b_s, seq_len, 2, num_attn_heads * head_dim)

recurrence_keys, recurrence_values = recurrence.unbind(dim=-2) # Each of the 2 has dim (b_s, seq_len, num_attn_heads * head_dim)

xl_keys = torch.cat((recurrence_keys, keys), dim=-2) # Prepend the recurrence keys to the keys along the seq_len dimension (-2)
# So we are basically doubling the seq_len

xl_values = torch.cat((recurrence_values, values), dim=-2)

print(queries.shape)  # torch.Size([16, 512, 80])
print(xl_keys.shape)  # torch.Size([16, 1024, 80])
xl_values.shape       # torch.Size([16, 1024, 80])






torch.Size([16, 512, 80])
torch.Size([16, 1024, 80])


torch.Size([16, 1024, 80])